In [1]:
from __future__ import print_function, division
%pylab inline
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings('ignore')
import seaborn as sns
from bokeh.plotting import *

from bokeh.io import output_notebook
output_notebook()
from collections import Counter
from operator import itemgetter

from termcolor import colored
import os
from keras.models import Sequential
from keras.layers.core import Activation, Dropout, Dense, Flatten, Lambda
from keras.layers.advanced_activations import LeakyReLU
from keras.layers.normalization import BatchNormalization
from keras.layers.embeddings import Embedding
from keras.layers.core import Reshape
from keras.layers.pooling import MaxPooling2D, AveragePooling2D
from keras.engine import Input
from keras.layers import Merge
from keras.layers import merge
from keras.layers import LSTM
from keras.models import Model
from keras.layers.convolutional import Convolution2D
from keras.optimizers import RMSprop, SGD
from keras.regularizers import l2, activity_l2, activity_l1, l1l2, l1
from keras.callbacks import EarlyStopping
from keras import backend as K
from tqdm import tqdm, tqdm_pandas
from collections import defaultdict
from nltk.corpus import stopwords
from keras.utils.np_utils import to_categorical

Populating the interactive namespace from numpy and matplotlib


Loading BokehJS ...

Using Theano backend.


In [2]:
df = pd.read_pickle('../data/tagged_resources.df')
print(df.columns)
df.head(2)

Index([               u'id',  u'course_shortname',               u'url',
             u'concept_tag',             u'title',          u'subtitle',
             u'description', u'short_description',             u'venue',
              u'slide_text',   u'transcript_text',              u'tags'],
      dtype='object')


,id,course_shortname,url,concept_tag,title,subtitle,description,short_description,venue,slide_text,transcript_text,tags
0,-4ffSHNYEeWIfhKr_WcYsQ,reproducible-res,http://www.coursera.org/learn/reproducible-res...,reproducibility,Reproducible Research,NaN,This course focuses on the concepts and tools ...,NaN,Coursera,NaN,NaN,"[reproducibility, project, r, in_depth, video_..."
1,-N44X0IJEeWpogr5ZO8qxQ,ml-clust-retrieve,http://www.coursera.org/learn/ml-clustering-an...,natural_language_processing,Machine Learning: Clustering & Retrieval,NaN,Case Studies: Finding Similar Documents\n\nA r...,NaN,Coursera,NaN,NaN,"[natural_language_processing, text_data, high_..."


In [3]:
class Digitizer:
    def __init__(self):
        self.word_dict = defaultdict(lambda: len(self.word_dict))
        self.word_dict[0] = ''
    def series_digitizer(self, text_series, max_len=-1):
        digitzed_ser = text_series.apply(self.digitize, args=(max_len,)).astype('object')
        max_len = digitzed_ser.apply(len).max()
        digitzed_ser = digitzed_ser.apply(lambda x: x if len(x) == max_len else x + [0] * (max_len - len(x)))
        return digitzed_ser
        
    def digitize(self, text, max_len=-1):
        if max_len < 0:
            return [self.word_dict[i] for i in text.split()]
        else:
            return [self.word_dict[i] for i in text.split()[:max_len]]
    def num_words(self):
        return len(self.word_dict)

In [4]:
import string
allowed_chars = set(string.ascii_lowercase) | {' '}
en_stopwords = set(stopwords.words("english"))
def text_cleanup(text):
    if isinstance(text, float):
        return ''
    text = text.lower()
    #text = ' '.join((stemmer.stem(i.decode('utf-8')) for i in text.split()))
    if text == 'nan':
        text = ''
    text = ''.join([i if i in allowed_chars else ' ' for i in text])
    text = ' '.join([i for i in text.split() if i not in en_stopwords])
    while '  ' in text:
        text = text.replace('  ', ' ')
    return text

In [5]:
df.columns

Index([               u'id',  u'course_shortname',               u'url',
             u'concept_tag',             u'title',          u'subtitle',
             u'description', u'short_description',             u'venue',
              u'slide_text',   u'transcript_text',              u'tags'],
      dtype='object')

In [7]:
digiz = Digitizer()
max_words = 10000
text_series = list()
for i in ['title', 'description', 'transcript_text']:
    ser = df[i].copy()
    ser = ser.apply(text_cleanup)
    print(i, ser.apply(len).max())
    text_series.append(digiz.series_digitizer(ser, max_len=max_words))
print('num distinct words:', digiz.num_words())

title 71
description 2104
transcript_text 0
num distinct words: 2708


In [8]:
text_series[0].head(2)

0    [1, 2, 0, 0, 0, 0, 0, 0, 0, 0]
1    [3, 4, 5, 6, 0, 0, 0, 0, 0, 0]
Name: title, dtype: object

In [9]:
text_series[1].head(2)

0    [107, 238, 45, 46, 151, 239, 240, 21, 241, 1, ...
1    [116, 117, 288, 289, 290, 291, 292, 293, 294, ...
Name: description, dtype: object

In [57]:
df['tags'] = df['tags'].apply(lambda x: list(filter(lambda y: y not in {'in_depth', 'mooc', 'video_lecture'}, x)))

In [58]:
all_text = text_series[0].copy()
for i in text_series[1:]:
    all_text += i
df['digitized_text'] = all_text
max_len = df['digitized_text'].apply(len).max()
print(max_len)
df['digitized_text'] = df['digitized_text'].apply(lambda x: np.array(x))
print(len(df))
df['digitized_text'].head(10)

286
148


0      [1, 2, 0, 0, 0, 0, 0, 0, 0, 0, 107, 238, 45, 4...
1      [3, 4, 5, 6, 0, 0, 0, 0, 0, 0, 116, 117, 288, ...
2      [7, 8, 9, 0, 0, 0, 0, 0, 0, 0, 107, 365, 121, ...
62     [10, 11, 12, 0, 0, 0, 0, 0, 0, 0, 391, 392, 39...
63     [13, 14, 15, 0, 0, 0, 0, 0, 0, 0, 510, 21, 442...
149    [16, 17, 18, 19, 20, 0, 0, 0, 0, 0, 394, 536, ...
150    [21, 22, 23, 0, 0, 0, 0, 0, 0, 0, 23, 64, 550,...
184    [24, 21, 25, 0, 0, 0, 0, 0, 0, 0, 21, 552, 564...
185    [3, 4, 26, 0, 0, 0, 0, 0, 0, 0, 116, 144, 582,...
186    [27, 28, 29, 30, 0, 0, 0, 0, 0, 0, 107, 629, 6...
Name: digitized_text, dtype: object

In [111]:
def create_class_vec(tag_series):
    tag2num = defaultdict(lambda : len(tag2num))
    tag_dig = tag_series.apply(lambda x: [tag2num[i] for i in set(x)])
    len_class_vec = len(tag2num)
    def tagdig2vec(x):
        v = np.zeros(len_class_vec)
        v[x] = 1.
        return v
    tag_dig = tag_dig.apply(tagdig2vec)
    return tag_dig, {j:i for i,j in tag2num.items()}

In [112]:
df['tagvec'], num2tag = create_class_vec(df['tags'])
df['tagvec'].head(2)
len_class_vec = len(df['tagvec'].iloc[0])
print(num2tag[3])

clustering


In [113]:
n_words = digiz.num_words()
print('#words:', n_words)
print('seq len:', max_len)
df['digitized_text'].head(2)

#words: 2708
seq len: 286


0    [1, 2, 0, 0, 0, 0, 0, 0, 0, 0, 107, 238, 45, 4...
1    [3, 4, 5, 6, 0, 0, 0, 0, 0, 0, 116, 117, 288, ...
Name: digitized_text, dtype: object

In [114]:
df[['tagvec', 'tags']].iloc[0]

tagvec    [1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...
tags                 [reproducibility, project, r, project]
Name: 0, dtype: object

In [115]:
X, y = np.vstack(df['digitized_text'].values), np.vstack(df['tagvec'])

In [126]:
def get_nn(n_words, input_len):   
    dim = 100
    emb = Embedding(input_dim=n_words, output_dim=dim, mask_zero=True)
    c = Input(shape=(input_len,), dtype='int32', name='course')
    #dropout = Dropout(0.5)
    c_e = emb(c)
    c_l = LSTM(128)(c_e)
    c_d = Dense(len_class_vec, activation='sigmoid')(c_l)
    model = Model(c, c_d)
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

In [127]:
nn = get_nn(n_words, max_len)
print(nn.summary())
#embedded_text = nn.predict(df['digitized_text'].values.tolist())

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
course (InputLayer)              (None, 286)           0                                            
____________________________________________________________________________________________________
embedding_13 (Embedding)         (None, 286, 3)        8124        course[0][0]                     
____________________________________________________________________________________________________
lstm_13 (LSTM)                   (None, 10)            560         embedding_13[0][0]               
____________________________________________________________________________________________________
dense_12 (Dense)                 (None, 71)            781         lstm_13[0][0]                    
Total params: 9465
________________________________________________________________________

In [128]:
hist = nn.fit(X, y,
              batch_size=8,
              validation_split=.1,
              shuffle=True,
              nb_epoch=100, 
              verbose=1)

data = zip(hist.history['loss'], hist.history['val_loss'])
hist_df = pd.DataFrame(columns=['train', 'val'], data=data[1:])
hist_df.plot(y=['train', 'val'], secondary_y=['val'])
plt.show()

Train on 133 samples, validate on 15 samples
Epoch 1/100
133/133 [==============================] - 1s - loss: 16.1584 - acc: 0.0301 - val_loss: 15.5945 - val_acc: 0.1333
Epoch 2/100
133/133 [==============================] - 1s - loss: 16.0465 - acc: 0.0150 - val_loss: 15.4860 - val_acc: 0.0000e+00
Epoch 3/100
133/133 [==============================] - 1s - loss: 15.7742 - acc: 0.0075 - val_loss: 15.3430 - val_acc: 0.0000e+00
Epoch 4/100
133/133 [==============================] - 1s - loss: 15.4565 - acc: 0.0075 - val_loss: 15.1875 - val_acc: 0.0000e+00
Epoch 5/100
133/133 [==============================] - 1s - loss: 15.0429 - acc: 0.0075 - val_loss: 15.0065 - val_acc: 0.0000e+00
Epoch 6/100
133/133 [==============================] - 1s - loss: 14.5722 - acc: 0.0075 - val_loss: 14.8616 - val_acc: 0.0000e+00
Epoch 7/100
133/133 [==============================] - 1s - loss: 14.1679 - acc: 0.0677 - val_loss: 14.8331 - val_acc: 0.0667
Epoch 8/100
133/133 [==============================] 

KeyboardInterrupt: 

In [89]:
tag_vec_pred = nn.predict(X)

In [90]:
def pred_to_tag(pred, th=.5):
    tags = list()
    idx = range(len(pred[0]))
    for i in pred:
        filt_preds = list(filter(lambda x: x[0] > th, zip(i, idx)))
        if len(filt_preds) > 0:
            tag_vals, tag_nums = zip(*filt_preds)
            pred_tags = map(lambda x: num2tag[x], tag_nums)
            tags.append(list(zip(pred_tags, tag_vals)))
        else:
            tags.append([tuple()])
        
    return tags

In [91]:
pred_tags = pred_to_tag(tag_vec_pred)
print(pred_tags[:5])

[[()], [()], [()], [()], [()]]


In [92]:
df['pred'] = 0
df['pred'] = df['pred'].astype('object')
df['pred'] = pred_tags

In [93]:
for ridx, (title, tags, pred) in df[['title', 'tags', 'pred']].iterrows():
    print(title)
    try:
        pred_dict = {i:j for i,j in pred}
    except ValueError:
        continue
    overlap = set(tags) & set(pred_dict.keys())
    print('\ttrue positives')
    for i in sorted(overlap):
        print('\t\t', i, pred_dict[i])
    print('\tfalse positives')
    for i in sorted(set(pred_dict.keys()) - set(tags)):
        print('\t\t', i, pred_dict[i])
    print('\tfalse negatives')
    for i in sorted(set(tags) - set(pred_dict.keys())):
        print('\t\t', i)    

Reproducible Research
Machine Learning: Clustering & Retrieval
Introduction to Genomic Technologies
Econometrics: Methods and Applications
Fundamentals of Quantitative Modeling
Measuring Causal Effects in the Social Sciences
Data Science Capstone
Developing Data Products
Machine Learning: Classification
Hadoop Platform and Application Framework
Practical Predictive Analytics: Models and Methods
Forensic Accounting and Fraud Examination
Python for Genomic Data Science
Customer Analytics
Data Management and Visualization
Business Intelligence Concepts, Tools, and Applications
Modeling Risk and Realities
Building a Data Science Team
Machine Learning With Big Data
Interprofessional Healthcare Informatics 
Statistics for Genomic Data Science
Decision-Making and Scenarios
Design and Build a Data Warehouse for Business Intelligence Implementation
Data Science at Scale - Capstone Project
Data Analysis Tools
Machine Learning: Recommender Systems & Dimensionality Reduction
Graph Analytics for Bi